<a href="https://colab.research.google.com/github/oliverwsd/finance/blob/master/ProblemSet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import pandas as pd
from io import StringIO
import csv
 
url = 'https://raw.githubusercontent.com/oliverwsd/finance/master/price_data.csv'
prices = pd.read_csv(url,parse_dates= ['Date'])
prices.head

<bound method NDFrame.head of            Date  VBTIX       GE       WMT       IBM      SPX
0    1999-12-31    NaN  49.5795   69.1250  107.8750  1469.25
1    2000-01-03    NaN  48.0577   66.8750  115.6250  1455.22
2    2000-01-04    NaN  46.1955   64.2500  112.0625  1399.42
3    2000-01-05    NaN  45.9552   63.0000  116.0000  1402.11
4    2000-01-06    NaN  46.6710   63.6875  115.0000  1403.45
...         ...    ...      ...       ...       ...      ...
5146 2019-09-23  84.25   9.3400  117.6200  142.0700  2991.78
5147 2019-09-24  84.54   8.9700  118.4000  141.6800  2966.60
5148 2019-09-25  84.14   9.1600  118.4700  143.1600  2984.87
5149 2019-09-26  84.28   9.0200  118.3000  143.5500  2977.62
5150 2019-09-27  84.37   9.0400  118.4500  143.2400  2961.79

[5151 rows x 6 columns]>

In [0]:
returns = prices.copy()
returns.loc[:, ['VBTIX', 'GE', 'WMT', 'IBM', 'SPX']] = np.log(returns.loc[:, ['VBTIX', 'GE', 'WMT', 'IBM', 'SPX']]) - np.log(returns.loc[:, ['VBTIX', 'GE', 'WMT', 'IBM', 'SPX']].shift())

avg_ret_annual = returns.loc[:, ['VBTIX', 'GE', 'WMT', 'IBM', 'SPX']].mean(axis = 0) * 252
std_annual = returns.loc[:, ['VBTIX', 'GE', 'WMT', 'IBM', 'SPX']].std(axis = 0, ddof = 0) * (252**0.5)


In [75]:
mu = np.matrix(avg_ret_annual).T
mu.shape   # Hint: The shape should be (5, 1)
mu

matrix([[ 0.0088722 ],
        [-0.08327833],
        [ 0.02635355],
        [ 0.0138746 ],
        [ 0.0343034 ]])

In [76]:
std_annual

VBTIX    0.044440
GE       0.307701
WMT      0.232419
IBM      0.253736
SPX      0.186346
dtype: float64

In [114]:
volatility_matrix = np.diag(std_annual)                  # This is the diagonal matrix of the volatilities
# correlation_matrix = returns.loc[:, ['VBTIX', 'GE', 'WMT', 'IBM', 'SPX']].corr() # Each row represents a variable
# corr_frame= np.array([returns['VBTIX'],
#           returns['GE'],
#           returns['WMT'],
#           returns['IBM'],
#           returns['SPX']])
# correlation_matrix = np.corrcoef(corr_frame)
# Cov = np.dot(volatility_matrix, correlation_matrix).dot(volatility_matrix)
Cov = returns.loc[:, ['VBTIX', 'GE', 'WMT', 'IBM', 'SPX']].corr()
Cov

,VBTIX,GE,WMT,IBM,SPX
VBTIX,1.000000,-0.187080,-0.059554,-0.156777,-0.190810
GE,-0.187080,1.000000,0.358308,0.461086,0.700323
WMT,-0.059554,0.358308,1.000000,0.322218,0.518941
IBM,-0.156777,0.461086,0.322218,1.000000,0.644094
SPX,-0.190810,0.700323,0.518941,0.644094,1.000000


In [131]:
# Task 2: Minimum Variance Portfolio
# ---------------------------------------

# Calculate the weights of the global minimum variance portfolio
# Mathematically, the solution for these weights is
#     w_mvp = (Cov^{-1} * e) / (transpose(e) * Cov^{-1} * e)
# where e is a (5, 1) vector of ones.
#
# Hint: To get the inverse (transpose) of a matrix use the '.getI()' ('.getT()') functions
#
e = np.mat(np.ones([5,1]))
transpose_e = e.getI()
w_mvp     = ((Cov**(-1)).dot(e))/(transpose_e.dot(Cov**(-1)).dot(e))

# Hint: Check for yourself: Do the weights of the minimum variance portfolio add up to 1?

# Calculate the expected mean of the minimum variance portfolio
# The expected mean of a portfolio with weight w is transpose(w) * mu
#
#mu_mvp    =                             # Make sure this is a scalar (i.e. a single number, not a 1x1 matrix)
#mu_mvp

Exception: ignored